In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Bert4Rec을 이용한 DKT 예측 모델 구현

In [2]:
import os
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import sigmoid
import wandb

import time
import pytz
import argparse
import math
from datetime import datetime
from typing import Tuple
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.preprocessing import LabelEncoder

## args 정리

In [3]:
def parse_args():
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--seed", default=42, type=int, help="seed")
    parser.add_argument("--device", default="cpu", type=str, help="cpu or gpu")

    parser.add_argument("--data_path", default="/opt/ml/input/data/", type=str, help="data directory")
    parser.add_argument("--asset_dir", default="asset/", type=str, help="data directory")
    parser.add_argument("--model_dir", default="models/", type=str, help="model directory")
    parser.add_argument(
        "--file_name", default="train_data.csv", type=str, help="train file name"
    )
    
    parser.add_argument("--num_workers", default=1, type=int, help="number of workers")


    # 훈련
    parser.add_argument("--n_epochs", default=20, type=int, help="number of epochs")
    parser.add_argument("--batch_size", default=64, type=int, help="batch size")
    parser.add_argument("--lr", default=0.0001, type=float, help="learning rate")
    parser.add_argument("--clip_grad", default=10, type=int, help="clip grad")
    parser.add_argument("--patience", default=5, type=int, help="for early stopping")

    parser.add_argument(
        "--log_steps", default=50, type=int, help="print log per n steps"
    )

    # BERT params - 개인적으로 하이퍼파라미터 튜닝
    parser.add_argument('--bert_max_len', type=int, default=13, help='Length of sequence for bert')
    parser.add_argument('--bert_num_items', type=int, default=9454, help='Number of total items') #assessmentid 수
    parser.add_argument('--bert_num_tags', type=int, default=912, help='Number of total items') #knowledgetag 수
    parser.add_argument('--bert_hidden_units', type=int, default=64, help='Size of hidden vectors (d_model)')
    parser.add_argument('--bert_num_blocks', type=int, default=2, help='Number of transformer layers')
    parser.add_argument('--bert_num_heads', type=int, default=2, help='Number of heads for multi-attention')
    parser.add_argument('--bert_dropout', type=float, default=0.2, help='Dropout probability to use throughout the model')
    parser.add_argument('--bert_mask_prob', type=float, default=0.1, help='Probability for masking items in the training sequence')

    # optimizer #
    parser.add_argument('--optimizer', type=str, default='Adam', choices=['Adam', 'AdamW'])
    parser.add_argument('--scheduler', type=str, default="plateau", help="scheduler type")
    parser.add_argument('--weight_decay', type=float, default=0, help='l2 regularization')
    parser.add_argument('--momentum', type=float, default=None, help='SGD momentum')
    # lr scheduler #
    parser.add_argument('--decay_step', type=int, default=15, help='Decay step for StepLR')
    parser.add_argument('--gamma', type=float, default=0.1, help='Gamma for StepLR')
    
    args = parser.parse_args('')
    return args

logger

In [4]:
def get_logger(logger_conf: dict):
    import logging
    import logging.config

    logging.config.dictConfig(logger_conf)
    logger = logging.getLogger()
    return logger

logging_conf = {  # only used when 'user_wandb==False'
    "version": 1,
    "formatters": {
        "basic": {"format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"}
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "INFO",
            "formatter": "basic",
            "stream": "ext://sys.stdout",
        },
        "file_handler": {
            "class": "logging.FileHandler",
            "level": "DEBUG",
            "formatter": "basic",
            "filename": "run.log",
        },
    },
    "root": {"level": "INFO", "handlers": ["console", "file_handler"]},
}


## data load and preprocess

In [5]:
#data path 설정
data_dir = "/opt/ml/input/data/"
train_path = "train_data.csv"
test_path = "test_data.csv"

In [6]:
class Preprocess_Bert:
    def __init__(self, args):
        self.args = args
        self.train_data = None
        self.test_data = None

    def get_train_data(self):
        return self.train_data

    def get_test_data(self):
        return self.test_data

    ### train / valid split을 위한 함수
    def split_data(self,
                   data: np.ndarray,
                   ratio: float = 0.8,
                   shuffle: bool = True,
                   seed: int = 0) -> Tuple[np.ndarray]:
        """
        split data into two parts with a given ratio.
        """
        if shuffle:
            random.seed(seed)  # fix to default seed 0
            random.shuffle(data)

        size = int(len(data) * ratio)
        data_1 = data[:size]
        data_2 = data[size:]
        return data_1, data_2

    def __save_labels(self, encoder: LabelEncoder, name: str) -> None:
        le_path = os.path.join(self.args.asset_dir, name + "_classes.npy")
        np.save(le_path, encoder.classes_)

    def __preprocessing(self, df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
        #범주형 변수 label encoding
        categories_lst = ["assessmentItemID", "testId", "KnowledgeTag"]

        #label saving을 위해서 필요
        if not os.path.exists(self.args.asset_dir):
            os.makedirs(self.args.asset_dir)

        for col in categories_lst:
            encoder = LabelEncoder()
            if is_train:
                # For UNKNOWN class
                cat = df[col].unique().tolist() + ["unknown"]
                encoder.fit(cat)
                self.__save_labels(encoder, col)
            else:
                label_path = os.path.join(self.args.asset_dir, col + "_classes.npy")
                encoder.classes_ = np.load(label_path)

                df[col] = df[col].apply(
                    lambda x: x if str(x) in encoder.classes_ else "unknown"
                )

            df[col] = df[col].astype(str)
            df[col] = encoder.transform(df[col])

        def convert_time(s: str):
            timestamp = time.mktime(
                datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
            )
            return int(timestamp)

        df["Timestamp"] = df["Timestamp"].apply(convert_time)

        # 같은 문제를 여러번 푼 경우 마지막만 반영되도록 정리

        # userid와 assessmentItemID 기준으로 그룹화한다.
        grouped = df.groupby(['userID', 'assessmentItemID'])
        #각 그룹별로 동일 문제를 몇 번 푸는지를 계산하고, 이를 맵핑할 딕셔너리를 만든다. ex) {(0, 'A020172001'): 1, ...}
        counts_dict = grouped.size().to_dict()
        # counts_dict를 이용하여 assessmentItemID별로 푼 문제 수를 맵핑한다.
        df['#ofsameSolved'] = df.set_index(['userID', 'assessmentItemID']).index.map(counts_dict) 

        df = df.sort_values(by=['userID', 'assessmentItemID', 'Timestamp'], ascending=[True, True, True])
        df = df.reset_index(drop=True)

        df.drop(df.loc[df['#ofsameSolved'] == 2].iloc[::2].index, axis = 0, inplace=True)  
        df.drop(df.loc[df['#ofsameSolved'] == 3].iloc[::3].index, axis = 0, inplace=True)
        df.drop(df.loc[df['#ofsameSolved'] == 3].iloc[::2].index, axis = 0, inplace=True)
        df.drop(columns='#ofsameSolved', axis = 1, inplace=True)

        return df

    # def __feature_engineering(self, df: pd.DataFrame) -> pd.DataFrame:
    #     
    #     return df

    def load_data_from_file(self, file_name: str, is_train: bool = True) -> np.ndarray:
        csv_file_path = os.path.join(self.args.data_path, file_name)
        df = pd.read_csv(csv_file_path)  # , nrows=100000)
        #df = self.__feature_engineering(df)
        df = self.__preprocessing(df, is_train)

        # 추후 feature를 embedding할 시에 embedding_layer의 input 크기를 결정할때 사용
        self.args.n_questions = len(
            np.load(os.path.join(self.args.asset_dir, "assessmentItemID_classes.npy"))
        )
        self.args.n_tests = len(
            np.load(os.path.join(self.args.asset_dir, "testId_classes.npy"))
        )
        self.args.n_tags = len(
            np.load(os.path.join(self.args.asset_dir, "KnowledgeTag_classes.npy"))
        )

        df = df.sort_values(by=["userID", "Timestamp"], axis=0)
        columns = ["userID", "assessmentItemID", "testId", "answerCode", "KnowledgeTag"]
        #userID와 testId로 groupby진행: 세션 단위 시퀀스 모델
        group = (
            df[columns]
            .groupby(["userID", "testId"])
            .apply(
                lambda r: (
                    r["assessmentItemID"].values,
                    r["KnowledgeTag"].values,
                    r["answerCode"].values,
                )
            )
        )
        return group.values

    def load_train_data(self, file_name: str) -> None:
        self.train_data = self.load_data_from_file(file_name)

    def load_test_data(self, file_name: str) -> None:
        self.test_data = self.load_data_from_file(file_name, is_train=False)

In [7]:
class DKTDataset_Bert(torch.utils.data.Dataset):
    def __init__(self, data: np.ndarray, args):
        self.data = data

    def __getitem__(self, index: int) -> dict:
        row = self.data[index]
        
        # Load from data : mask된 값을 0으로 처리하기 위해 1을 더해줌
        question, tag, correct = row[0], row[1], row[2]
        data = {
            "question": torch.tensor(question + 1, dtype=torch.int),
            "tag": torch.tensor(tag + 1, dtype=torch.int),
            "correct": torch.tensor(correct, dtype=torch.int),
        }

        # Generate mask 
        seq_len = len(row[0])
        for k, seq in data.items():
            # Pre-padding non-valid sequences
            tmp = torch.zeros(13) #하나의 시험지의 최대 길이 13
            tmp[13-seq_len:] = data[k]
            data[k] = tmp
        mask = torch.zeros(13, dtype=torch.int16)
        mask[-seq_len:] = 1
        data["mask"] = mask
        
        # Generate interaction
        interaction = data["correct"] + 1  # 패딩을 위해 correct값에 1을 더해준다.
        interaction = interaction.roll(shifts=1)
        interaction_mask = data["mask"].roll(shifts=1)
        interaction_mask[0] = 0
        interaction = (interaction * interaction_mask).to(torch.int64)
        data["interaction"] = interaction
        data = {k: v.int() for k, v in data.items()}

        return data

    def __len__(self) -> int:
        return len(self.data)

In [8]:
def get_loaders(args, train: np.ndarray, valid: np.ndarray) -> Tuple[torch.utils.data.DataLoader]:
    pin_memory = False
    train_loader, valid_loader = None, None

    if train is not None:
        trainset = DKTDataset_Bert(train, args)
        train_loader = torch.utils.data.DataLoader(
            trainset,
            num_workers=args.num_workers,
            shuffle=True,
            batch_size=args.batch_size,
            pin_memory=pin_memory,
        )
    if valid is not None:
        valset = DKTDataset_Bert(valid, args)
        valid_loader = torch.utils.data.DataLoader(
            valset,
            num_workers=args.num_workers,
            shuffle=False,
            batch_size=args.batch_size,
            pin_memory=pin_memory,
        )

    return train_loader, valid_loader

In [9]:
args = parse_args()
preprocess = Preprocess_Bert(args)
preprocess.load_train_data(file_name=args.file_name)
train_data: np.ndarray = preprocess.get_train_data()
train_data, valid_data = preprocess.split_data(data=train_data)

In [10]:
len(train_data)

292131

In [11]:
len(valid_data)

73033

In [12]:
train_data

array([(array([8891, 8892, 8893, 8894, 8895, 8896, 8897, 8898]), array([768, 768, 769, 767, 767, 770, 770, 770]), array([1, 0, 0, 0, 0, 0, 1, 0])),
       (array([3532, 3533, 3534, 3535, 3536]), array([205, 205, 205, 205, 205]), array([1, 0, 0, 0, 0])),
       (array([1525, 1526, 1528, 1527, 1529]), array([724, 724, 724, 724, 724]), array([1, 1, 1, 1, 0])),
       ...,
       (array([3948, 3949, 3950, 3951, 3952]), array([229, 228, 228, 229, 229]), array([1, 1, 1, 1, 1])),
       (array([616, 617, 618, 619, 620]), array([601, 601, 601, 601, 601]), array([1, 1, 1, 1, 1])),
       (array([8939, 8940, 8941, 8942, 8943, 8944, 8945, 8946]), array([426, 426,   7, 671, 671, 671, 426, 426]), array([0, 1, 1, 1, 1, 1, 1, 0]))],
      dtype=object)

## bert4rec 모델 만들기

### embedding layer

In [13]:
#bert token embedding
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

#bert positional embedding
class PositionalEmbedding(nn.Module):

    def __init__(self, max_len, d_model):
        super().__init__()

        # Compute the positional encodings once in log space.
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size = x.size(0)
        return self.pe.weight.unsqueeze(0).repeat(batch_size, 1, 1)
    
class BERTEmbedding(nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos

        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, max_len, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(max_len=max_len, d_model=embed_size)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)
        return self.dropout(x)

### transformer layer
1. multihead attention

In [14]:
#single attention : attention 수식 구현
class Attention(nn.Module):
    """
    Compute 'Scaled Dot Product Attention
    """

    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn
    

#multihead attention
class MultiHeadedAttention(nn.Module):
    """
    Take in model size and number of heads.
    """

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linear_layers, (query, key, value))]
        # 2) Apply attention on all the projected vectors in batch.
        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)

2. pointwise feedforward layer

In [15]:
#bert uses gelu instead of relu
class GELU(nn.Module):

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))
    

class LayerNorm(nn.Module):

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
#residual connection
class SublayerConnection(nn.Module):

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

transformer block 구현

In [16]:
class TransformerBlock(nn.Module):
    """
    Bidirectional Encoder = Transformer (self-attention)
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        """
        :param hidden: hidden size of transformer
        :param attn_heads: head sizes of multi-head attention
        :param feed_forward_hidden: feed_forward_hidden, usually 4*hidden_size
        :param dropout: dropout rate
        """
        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)

### bert 구현

In [17]:
def set_seeds(seed: int = 42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [18]:
class BERT(nn.Module):
    def __init__(self, args):
        super().__init__()

        set_seeds(args.seed)
        # self.init_weights()
        
        max_len = args.bert_max_len
        num_items = args.bert_num_items
        n_tags = args.bert_num_tags
        n_layers = args.bert_num_blocks
        heads = args.bert_num_heads
        hidden = args.bert_hidden_units
        self.hidden = hidden
        dropout = args.bert_dropout

        # embedding for BERT, sum of positional, segment, token embeddings

        # self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=self.hidden, max_len=max_len, dropout=dropout)
        # hd, intd = hidden, hidden // 3
        # self.embedding_interaction = nn.Embedding(3, intd) # interaction은 현재 correct로 구성되어있다. correct(1, 2) + padding(0)
        # self.embedding_question = nn.Embedding(num_items + 1, intd)
        # self.embedding_tag = nn.Embedding(n_tags + 1, intd)

        self.embedding_interaction = BERTEmbedding(vocab_size=3, embed_size=self.hidden, max_len = max_len, dropout=dropout)
        self.embedding_question = BERTEmbedding(vocab_size=num_items + 1, embed_size=self.hidden, max_len = max_len, dropout=dropout)
        self.embedding_tag = BERTEmbedding(vocab_size=n_tags + 1, embed_size=self.hidden, max_len = max_len, dropout=dropout)

        # Concatentaed Embedding Projection
        self.comb_proj = nn.Linear(self.hidden * 3, self.hidden)

        # Fully connected layer
        self.fc = nn.Linear(self.hidden, 1)

        # multi-layers transformer blocks, deep network
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden, heads, hidden * 4, dropout) for _ in range(n_layers)])

    ## 수정 ##
    def forward(self, question, tag, correct, mask, interaction):
        batch_size = interaction.size(0)
        # Embedding
        embed_interaction = self.embedding_interaction(interaction.int())
        embed_question = self.embedding_question(question.int())
        embed_tag = self.embedding_tag(tag.int())
        embed = torch.cat(
            [
                embed_interaction,
                embed_question,
                embed_tag,
            ],
            dim=2,
        )
        
        X = self.comb_proj(embed)

        mask = mask.unsqueeze(1).repeat(1, X.size(1), 1).unsqueeze(1)

        # running over multiple transformer blocks
        for transformer in self.transformer_blocks:
            X = transformer.forward(X, mask)

        encoded_layers = X
        # out = encoded_layers[0]
        # out = out.contiguous().view(batch_size, -1, self.hidden)

        out = encoded_layers
        out = self.fc(out).view(batch_size, -1)
        return out

    def init_weights(self):
        pass

In [19]:
class BERTModel(nn.Module):
    def __init__(self, args):
        super().__init__(args)
        self.args = args
        self.bert = BERT(args)
        self.out = nn.Linear(self.bert.hidden, args.num_items + 1)

    def code(cls):
        return 'bert'

    def forward(self, x):
        x = self.bert(x)
        return self.out(x)

## train bert model

optimzer, scheduler 설정

In [20]:
from torch.optim import Adam, AdamW


def get_optimizer(model: torch.nn.Module, args):
    if args.optimizer == "Adam":
        optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=0.01)
        
    elif args.optimizer == "AdamW":
        optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=0.01)
    # 모든 parameter들의 grad값을 0으로 초기화
    optimizer.zero_grad()
    return optimizer


In [21]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_linear_schedule_with_warmup


def get_scheduler(optimizer: torch.optim.Optimizer, args):
    if args.scheduler == "plateau":
        scheduler = ReduceLROnPlateau(
            optimizer, patience=10, factor=0.5, mode="max", verbose=True
        )
    elif args.scheduler == "linear_warmup":
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=args.total_steps,
        )
    return scheduler

train, vaild 실행

In [22]:
logger = get_logger(logger_conf=logging_conf)

def run(args,
        train_data: np.ndarray,
        valid_data: np.ndarray,
        model: nn.Module):
    train_loader, valid_loader = get_loaders(args=args, train=train_data, valid=valid_data)

    # For warmup scheduler which uses step interval
    args.total_steps = int(math.ceil(len(train_loader.dataset) / args.batch_size)) * (
        args.n_epochs
    )
    args.warmup_steps = args.total_steps // 10

    optimizer = get_optimizer(model=model, args=args)
    scheduler = get_scheduler(optimizer=optimizer, args=args)

    best_auc = -1
    early_stopping_counter = 0
    for epoch in range(args.n_epochs):
        logger.info("Start Training: Epoch %s", epoch + 1)

        # TRAIN
        train_auc, train_acc, train_loss = train(train_loader=train_loader,
                                                 model=model, optimizer=optimizer,
                                                 scheduler=scheduler, args=args)

        # VALID
        auc, acc = validate(valid_loader=valid_loader, model=model, args=args)

        # wandb.log(dict(epoch=epoch,
        #                train_loss_epoch=train_loss,
        #                train_auc_epoch=train_auc,
        #                train_acc_epoch=train_acc,
        #                valid_auc_epoch=auc,
        #                valid_acc_epoch=acc))
        
        if auc > best_auc:
            best_auc = auc
            # nn.DataParallel로 감싸진 경우 원래의 model을 가져옵니다.
            model_to_save = model.module if hasattr(model, "module") else model
            save_checkpoint(state={"epoch": epoch + 1,
                                   "state_dict": model_to_save.state_dict()},
                            model_dir=args.model_dir,
                            model_filename="best_model.pt")
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= args.patience:
                logger.info(
                    "EarlyStopping counter: %s out of %s",
                    early_stopping_counter, args.patience
                )
                break

        # scheduler
        if args.scheduler == "plateau":
            scheduler.step(best_auc)

In [23]:
def save_checkpoint(state: dict, model_dir: str, model_filename: str) -> None:
    """ Saves checkpoint to a given directory. """
    save_path = os.path.join(model_dir, model_filename)
    logger.info("saving model as %s...", save_path)
    os.makedirs(model_dir, exist_ok=True)
    torch.save(state, save_path)

In [24]:
def compute_loss(preds: torch.Tensor, targets: torch.Tensor):
    """
    loss계산하고 parameter update
    Args :
        preds   : (batch_size, max_seq_len)
        targets : (batch_size, max_seq_len)

    """
    loss = get_criterion(pred=preds, target=targets.float())

    # 마지막 시퀀드에 대한 값만 loss 계산
    loss = loss[:, -1]
    loss = torch.mean(loss)
    return loss

def get_criterion(pred: torch.Tensor, target: torch.Tensor):
    loss = torch.nn.BCEWithLogitsLoss(reduction="none")
    return loss(pred, target)

In [25]:
def update_params(loss: torch.Tensor,
                  model: nn.Module,
                  optimizer: torch.optim.Optimizer,
                  scheduler: torch.optim.lr_scheduler._LRScheduler,
                  args):
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)
    if args.scheduler == "linear_warmup":
        scheduler.step()
    optimizer.step()
    optimizer.zero_grad()

In [26]:
def get_metric(targets: np.ndarray, preds: np.ndarray) -> Tuple[float]:
    auc = roc_auc_score(y_true=targets, y_score=preds)
    acc = accuracy_score(y_true=targets, y_pred=np.where(preds >= 0.5, 1, 0))
    return auc, acc

In [27]:
def train(train_loader: torch.utils.data.DataLoader,
          model: nn.Module,
          optimizer: torch.optim.Optimizer,
          scheduler: torch.optim.lr_scheduler._LRScheduler,
          args):
    model.train()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in enumerate(train_loader):
        batch = {k: v.to(args.device) for k, v in batch.items()}
        preds = model(**batch)
        targets = batch["correct"]
        
        loss = compute_loss(preds=preds, targets=targets)
        update_params(loss=loss, model=model, optimizer=optimizer,
                      scheduler=scheduler, args=args)

        if step % args.log_steps == 0:
            logger.info("Training steps: %s Loss: %.4f", step, loss.item())

        # predictions
        preds = sigmoid(preds[:, -1])
        targets = targets[:, -1]

        total_preds.append(preds.detach())
        total_targets.append(targets.detach())
        losses.append(loss)

    total_preds = torch.concat(total_preds).cpu().numpy()
    total_targets = torch.concat(total_targets).cpu().numpy()

    # Train AUC / ACC
    auc, acc = get_metric(targets=total_targets, preds=total_preds)
    loss_avg = sum(losses) / len(losses)
    logger.info("TRAIN AUC : %.4f ACC : %.4f", auc, acc)
    return auc, acc, loss_avg

In [28]:
def validate(valid_loader: nn.Module, model: nn.Module, args):
    model.eval()

    total_preds = []
    total_targets = []
    for step, batch in enumerate(valid_loader):
        batch = {k: v.to(args.device) for k, v in batch.items()}
        preds = model(**batch)
        targets = batch["correct"]

        # predictions
        preds = sigmoid(preds[:, -1])
        targets = targets[:, -1]

        total_preds.append(preds.detach())
        total_targets.append(targets.detach())

    total_preds = torch.concat(total_preds).cpu().numpy()
    total_targets = torch.concat(total_targets).cpu().numpy()

    # Train AUC / ACC
    auc, acc = get_metric(targets=total_targets, preds=total_preds)
    logger.info("VALID AUC : %.4f ACC : %.4f", auc, acc)
    return auc, acc

In [29]:
def inference(args, test_data: np.ndarray, model: nn.Module) -> None:
    model.eval()
    _, test_loader = get_loaders(args=args, train=None, valid=test_data)

    total_preds = []
    for step, batch in enumerate(test_loader):
        batch = {k: v.to(args.device) for k, v in batch.items()}
        preds = model(**batch)

        # predictions
        preds = sigmoid(preds[:, -1])
        preds = preds.cpu().detach().numpy()
        total_preds += list(preds)

    write_path = os.path.join(args.output_dir, "submission.csv")
    os.makedirs(name=args.output_dir, exist_ok=True)
    with open(write_path, "w", encoding="utf8") as w:
        w.write("id,prediction\n")
        for id, p in enumerate(total_preds):
            w.write("{},{}\n".format(id, p))
    logger.info("Successfully saved submission as %s", write_path)


In [30]:
def save_checkpoint(state: dict, model_dir: str, model_filename: str) -> None:
    """ Saves checkpoint to a given directory. """
    save_path = os.path.join(model_dir, model_filename)
    logger.info("saving model as %s...", save_path)
    os.makedirs(model_dir, exist_ok=True)
    torch.save(state, save_path)

In [31]:
def load_model(args):
    # model_path = os.path.join(args.model_dir)
    # logger.info("Loading Model from: %s", model_path)
    # load_state = torch.load(model_path)
    model = BERT(args) #이 부분을 내가 만든 bert를 가져오도록

    # load model state
    # model.load_state_dict(load_state["state_dict"], strict=True)
    # logger.info("Successfully loaded model state from: %s", model_path)
    return model


In [32]:
# wandb.login()

# wandb.init(project="dkt", config=vars(args))

model: torch.nn.Module = load_model(args=args).to(args.device)

In [33]:
run(args,train_data,valid_data, model)

2023-05-23 23:46:20,916 - root - INFO - Start Training: Epoch 1


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.IntTensor instead (while checking arguments for embedding)